<div style="background-color: steelblue; padding: 10px; border-radius: 5px;">
    <p style="margin: 10;"></p>
    <h1 style="text-align: center; margin: 0; font-weight: bold; color: white;">SWOT Ocean Model Intercomparison Project : EORCA12-ERA5v1 Test Case</h1>
    <p style="margin: 10;"></p>
</div>



## 📦**Imports** 

In [1]:
%%time

##################################
#Imports

from datetime import datetime
import s3fs
import xarray as xr
import pyinterp
from widetrax import DataPreprocessing as dp
from widetrax import Spectra as sp
import numpy as np
import json
from watermark import watermark
import time
import platform

CPU times: user 3.45 s, sys: 515 ms, total: 3.96 s
Wall time: 5.36 s


## 🔍 **Check the S3 Endpoint**  

In [2]:
s3_folder ="s3://npd-eorca12-era5v1-swot-omip/"
endpoint_url =  "https://noc-msm-o.s3-ext.jc.rl.ac.uk"
fs = s3fs.S3FileSystem(anon=True,endpoint_url=endpoint_url)
# List contents of the bucket
bucket_name = 's3://npd-eorca12-era5v1-swot-omip/'
contents = fs.ls(bucket_name)
print("Bucket contents:")
for item in contents:
    print(item)

Bucket contents:
npd-eorca12-era5v1-swot-omip/cycle_008
npd-eorca12-era5v1-swot-omip/cycle_009
npd-eorca12-era5v1-swot-omip/cycle_010
npd-eorca12-era5v1-swot-omip/cycle_011
npd-eorca12-era5v1-swot-omip/cycle_012
npd-eorca12-era5v1-swot-omip/cycle_013
npd-eorca12-era5v1-swot-omip/cycle_014
npd-eorca12-era5v1-swot-omip/cycle_015
npd-eorca12-era5v1-swot-omip/cycle_016
npd-eorca12-era5v1-swot-omip/cycle_017
npd-eorca12-era5v1-swot-omip/cycle_018
npd-eorca12-era5v1-swot-omip/cycle_019
npd-eorca12-era5v1-swot-omip/cycle_020
npd-eorca12-era5v1-swot-omip/cycle_021
npd-eorca12-era5v1-swot-omip/cycle_022
npd-eorca12-era5v1-swot-omip/cycle_023
npd-eorca12-era5v1-swot-omip/cycle_024
npd-eorca12-era5v1-swot-omip/cycle_025
npd-eorca12-era5v1-swot-omip/cycle_026


## 🧮 **Required variables**

In [3]:
# "region_id": 5
lon_min = 114.87,
lon_max = 122.87,
lat_min = -40.5,
lat_max = -30.5,
       
area = [lon_min, lat_min, lon_max, lat_max]
start_date = "01012024" # "DDMMYYYY"
end_date ="30032024"

## 🔄 **Identify the cycle numbers within the specified time range** 

In [4]:
file_path = "https://minio.lab.dive.edito.eu/project-meom-ige/cycles_periods.csv"
matching_cycles = dp.get_matching_cycles(file_path, start_date, end_date)

def formater_numeros_concis(liste_numeros):
  return [str(numero).zfill(3) for numero in liste_numeros]
    
matching_cycles = formater_numeros_concis(matching_cycles)
matching_cycles

['008', '009', '010', '011', '012', '013']

## 📊 **Load datasets and compute Wavenumber spectra**

In [5]:
%%time
# Load datasets into a dictionary
datasets_dict = dp.read_swot_ncfiles_S3subfolders(
    s3_folder,
    matching_cycles,
    endpoint_url,
    area)

H_008_5 included.
H_008_5 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_1 included.
H_009_2 included.
H_009_2 included.
H_009_2 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_5 included.
H_009_5 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_1 included.
H_010_2 included.
H_010_2 included.
H_010_2 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_5 included.
H_010_5 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_1 included.
H_011_2 included.
H_011_2 included.
H_011_2 included.
H_011_3 included.
H_011_3 included.
H_011_3 included.
H_011_3 in

In [6]:
%%time
has_converged, filled_datasets = dp.fill_nan(datasets_dict, varname = "ssh")

CPU times: user 31.1 s, sys: 46 s, total: 1min 17s
Wall time: 30.2 s


In [7]:
%%time
segments_dict = sp.retrieve_segments(filled_datasets,FileType = "NetCDF",namevar="ssh")

/home/jovyan/conda-envs/amine_env/lib/python3.9/site-packages/widetrax/Spectra.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for col in range(dataset.dims['num_pixels']):


CPU times: user 712 ms, sys: 0 ns, total: 712 ms
Wall time: 712 ms


In [8]:
%%time
psd_dict, freqs_dict = sp.calculate_psd(segments_dict)
# Calculate PSD Mean
psd_mean, freqs_mean = sp.psd_mean_and_freq(psd_dict,freqs_dict)

CPU times: user 825 ms, sys: 425 ms, total: 1.25 s
Wall time: 1.21 s


## 💾 **Save Results and Information in JSON File**

In [9]:
# Infos
nom_region = "Region_5"
model = "eorca12-era5v1"
freqs_mean = np.array(freqs_mean)
psd_mean = np.array(psd_mean)
todays_Date = datetime.now()
date_creation = todays_Date.isoformat() #ISO 8601 format
packages_versions = watermark(packages="numpy,widetrax,xarray,s3fs,pyinterp,datetime,json,platform,time")
machine_info = {
    "system": platform.system(),
    "node_name": platform.node(),
    "release": platform.release(),
    "version": platform.version(),
    "machine": platform.machine(),
    "processor": platform.processor(),
    "python_version": platform.python_version()
}

# JSON file structure
donnees = {
    "region": nom_region,
    "model": model,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "start_date": datetime.strptime(start_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "end_date": datetime.strptime(end_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "packages_versions": packages_versions,
    "date_creation": date_creation,
    "login_creator": platform.node(),
    "Frequency": freqs_mean.tolist(),
    "Spectra": psd_mean.tolist(),
    "machine_info": machine_info
}

# Save on JSON file
with open("OMIP_"+str(model)+"_"+str(nom_region)+".json", "w", encoding="utf-8") as f:
    json.dump(donnees, f, ensure_ascii=False, indent=4)

print("JSON file created")

JSON file created


## 📤 **Export Results to the S3 Endpoint** 

In [16]:
fs = s3fs.S3FileSystem( anon=True, endpoint_url="https://minio.lab.dive.edito.eu", use_ssl=False ) 

In [15]:
json_file = "OMIP_"+str(model)+"_"+str(nom_region)+".json"
fs.put(json_file, "project-meom-ige/OMIP/")

[None]